In [5]:
import fiona
import geopandas as gpd
import pandas as pd

# Step 1: Read the shapefile
shapefile_path = 'h:\\Yehmh\\FuShan\\Fushan_seg_labeled\\202403_rgb_masks_cuda_noEdge_2up.shp'  # replace with your actual file path
# gdf = gpd.read_file(shapefile_path)

with fiona.Env(SHAPE_RESTORE_SHX=True):
    gdf = gpd.read_file(shapefile_path)

# Step 2: Filter polygons with the label 'f'
filtered_gdf = gdf[gdf['mnemonic'] == 'f']  # replace 'mnemonics' with your actual column name if different

# Step 3: Calculate the center (centroid) of each polygon
filtered_gdf['centroid'] = filtered_gdf.geometry.centroid

# Step 4: Extract the coordinates of the centroids
centroids = filtered_gdf['centroid']
centroid_coords = [(point.x, point.y) for point in centroids]

# Step 5: Create a DataFrame with the centroid coordinates
centroid_df = pd.DataFrame(centroid_coords, columns=['x', 'y'])

# Step 6: Save the DataFrame to a CSV file
csv_output_path = 'h:\\Yehmh\\FuShan\\FS_flower_centroids.csv'  # replace with your desired output file path
centroid_df.to_csv(csv_output_path, index=False)

print(f"Centroid coordinates saved to {csv_output_path}")


c:\Users\user\.conda\envs\RS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Centroid coordinates saved to h:\Yehmh\FuShan\FS_flower_centroids.csv


In [25]:
import pandas as pd
import numpy as np

tm2_00_x = 307002.7185
tm2_00_y = 2738993.585

centroids_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_flower_centroids.csv'
all_trees_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_all_trees_no0dbh.csv'
centroid_df = pd.read_csv(centroids_csv_file_path)
all_trees_df = pd.read_csv(all_trees_csv_file_path)

centroid_df['x'] = centroid_df['x'] - tm2_00_x
centroid_df['y'] = centroid_df['y'] - tm2_00_y

# Filter trees
filtered_trees_df = all_trees_df[all_trees_df['dbh'] > 20]

# Function to calculate the Euclidean distance
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Find trees that are within 10 meters from any centroid
trees_within_10m = []

for index, centroid in centroid_df.iterrows():
    distances = filtered_trees_df.apply(lambda tree: euclidean_distance(centroid['x'], centroid['y'], tree['plotx'], tree['ploty']), axis=1)
    close_trees = filtered_trees_df[distances <= 20].copy()
    trees_within_10m.append(close_trees)

# Combine the results into a single DataFrame
trees_within_10m_df = pd.concat(trees_within_10m, ignore_index=True)

trees_within_10m_df['tm2_x'] = trees_within_10m_df['plotx'] + tm2_00_x 
trees_within_10m_df['tm2_y'] = trees_within_10m_df['ploty'] + tm2_00_y

print(trees_within_10m_df)

output_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_flower_trees_within_20m.csv'
trees_within_10m_df.to_csv(output_csv_file_path, index=False)

# ----------------------------------------------------------------
# Count the number of each tree species within 10 meters
# ----------------------------------------------------------------

# Calculate the number of each tree species within 10 meters
species_count = trees_within_10m_df['mnemonic'].value_counts().reset_index()
species_count.columns = ['mnemonic', 'count']

# Save the results to a new CSV file (optional)
# output_csv_file_path = 'h:\\Yehmh\\FuShan\\species_count_within_10m.csv'
# species_count.to_csv(output_csv_file_path, index=False)

print(species_count)

     mnemonic   plotx   ploty   dbh        tm2_x        tm2_y
0      CASTCU  284.10  334.37  29.3  307286.8185  2739327.955
1      CASTCU  284.10  334.37  32.3  307286.8185  2739327.955
2      CASTCU  284.10  334.37  35.5  307286.8185  2739327.955
3      CASTCU  284.10  334.37  37.9  307286.8185  2739327.955
4      CASTCU  285.16  335.82  20.4  307287.8785  2739329.405
...       ...     ...     ...   ...          ...          ...
8208   PYRESH  212.35  167.45  20.1  307215.0685  2739161.035
8209   SCHEOC  204.48  143.28  20.3  307207.1985  2739136.865
8210   SCHEOC  204.48  143.28  22.8  307207.1985  2739136.865
8211   SCHEOC  204.48  143.28  25.0  307207.1985  2739136.865
8212   SCHEOC  205.59  145.85  20.6  307208.3085  2739139.435

[8213 rows x 6 columns]
   mnemonic  count
0    LIMLUR   1540
1    CASTCU   1431
2    MELISQ   1072
3    MACHTH    677
4    LITSAC    621
5    ELAEJA    464
6    PYRESH    444
7    MACHMU    306
8    DIOSMO    210
9    ENGERO    200
10   CINNMI    160
11 

In [24]:
import pandas as pd
import numpy as np

tm2_00_x = 307002.7185
tm2_00_y = 2738993.585

centroids_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_flower_centroids.csv'
all_trees_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_all_trees_no0dbh.csv'
centroid_df = pd.read_csv(centroids_csv_file_path)
all_trees_df = pd.read_csv(all_trees_csv_file_path)

centroid_df['x'] = centroid_df['x'] - tm2_00_x
centroid_df['y'] = centroid_df['y'] - tm2_00_y

# Filter trees
filtered_trees_df = all_trees_df[all_trees_df['dbh'] > 20]

# Function to calculate the Euclidean distance
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Find trees that are within 10 meters from any centroid
trees_within_10m = []
species_within_10m = []

for index, centroid in centroid_df.iterrows():
    distances = filtered_trees_df.apply(lambda tree: euclidean_distance(centroid['x'], centroid['y'], tree['plotx'], tree['ploty']), axis=1)
    close_trees = filtered_trees_df[distances <= 20].copy()
    trees_within_10m.append(close_trees)
    species_list = close_trees['mnemonic'].unique().tolist()
    species_within_10m.append(set(species_list))

# # Combine the results into a single DataFrame
# trees_within_10m_df = pd.concat(trees_within_10m, ignore_index=True)

# trees_within_10m_df['tm2_x'] = trees_within_10m_df['plotx'] + tm2_00_x 
# trees_within_10m_df['tm2_y'] = trees_within_10m_df['ploty'] + tm2_00_y

# print(trees_within_10m_df)

# output_csv_file_path = 'h:\\Yehmh\\FuShan\\FS_flower_trees_within_20m.csv'
# trees_within_10m_df.to_csv(output_csv_file_path, index=False)

# Find species that occur near every centroid
if species_within_10m:
    common_species = set.intersection(*species_within_10m)
else:
    common_species = set()

# Convert the results into a DataFrame
common_species_df = pd.DataFrame(list(common_species), columns=['mnemonic'])

print(common_species_df)

# ----------------------------------------------------------------
# Count the number of each tree species within 10 meters
# ----------------------------------------------------------------

# Calculate the number of each tree species within 10 meters
species_count = trees_within_10m_df['mnemonic'].value_counts().reset_index()
species_count.columns = ['mnemonic', 'count']

# Save the results to a new CSV file (optional)
# output_csv_file_path = 'h:\\Yehmh\\FuShan\\species_count_within_10m.csv'
# species_count.to_csv(output_csv_file_path, index=False)

print(species_count)

  mnemonic
0   LIMLUR
1   CASTCU
   mnemonic  count
0    CASTCU    108
1    LIMLUR     92
2    MACHTH     83
3    LITSAC     57
4    MELISQ     57
5    MACHMU     38
6    PYRESH     31
7    DIOSMO     30
8    ELAEJA     27
9    SCHEOC     26
10   MYRSSE     13
11   PRUNPH     11
12   CINNMI      8
13   RHUSSU      4
14   SYMPGL      4
15   GORDAX      2
16   CYATPO      2
17   SYMPTH      2
18   ENGERO      2
19   GLOCAC      1
